# depp learning

# 如何计算accuracy

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [24]:
logits = torch.rand(4,10)
pred = F.softmax(logits, dim = 1)
pred_label = pred.argmax(dim = 1)
print(pred_label)

label = torch.tensor([9,3,2,4])
correct = torch.eq(pred_label, label)

acc = correct.sum().float().item()/4
print(acc)

tensor([4, 3, 2, 2])
0.5


*测试acc一般选择一个epoch测一次*

## Conv2d

In [30]:
layer = nn.Conv2d(1, 3, kernel_size = 3, stride = 2, padding = 1)
x = torch.rand(1, 1, 28, 28)
out = layer(x)
#out = layer.forward(x) :不推荐使用
out.shape

torch.Size([1, 3, 14, 14])

Conv2d(input_channel, output_channel, kernel_size, stride, padding)

In [9]:
layer.weight.shape

torch.Size([3, 1, 3, 3])

In [10]:
layer.bias.shape

torch.Size([3])

最大池化

In [21]:
x = out
pool_layer = nn.MaxPool2d(2,stride = 2)
out = pool_layer(x)
out.shape

torch.Size([1, 3, 3, 3])

上采样

In [31]:
x = out
print(x.shape)
out = F.interpolate(x, scale_factor = 2, mode = 'nearest')
out.shape

torch.Size([1, 3, 14, 14])


torch.Size([1, 3, 28, 28])

ReLU

In [32]:
x = out
relu_layer = nn.ReLU(inplace = True)
out = relu_layer(x)
out.shape

torch.Size([1, 3, 28, 28])

批量归一化BN

In [39]:
x = torch.rand(100, 16, 784)
layer = nn.BatchNorm1d(16) #和通道数对应
out = layer(x)
print(layer.running_mean)
print(layer.running_var)
print(out.mean())#0
print(out.var())#1

tensor([0.0499, 0.0501, 0.0501, 0.0501, 0.0500, 0.0500, 0.0501, 0.0499, 0.0500,
        0.0500, 0.0501, 0.0499, 0.0499, 0.0501, 0.0501, 0.0500])
tensor([0.9083, 0.9083, 0.9083, 0.9083, 0.9083, 0.9083, 0.9083, 0.9083, 0.9084,
        0.9084, 0.9083, 0.9083, 0.9083, 0.9083, 0.9083, 0.9083])
tensor(6.8363e-09, grad_fn=<MeanBackward0>)
tensor(0.9999, grad_fn=<VarBackward0>)


In [44]:
x = torch.rand(6,3, 28, 28)
layer = nn.BatchNorm2d(3)
out = layer(x)
print(layer.weight)
print(layer.weight.shape)

Parameter containing:
tensor([1., 1., 1.], requires_grad=True)
torch.Size([3])


残差网络

In [61]:
class ResBlk(nn.Module):
    def __init__(self, ch_in, ch_out):
        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size = 3, stride = 1, padding = 1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size = 3, stride = 1, padding = 1)
        self.bn2 = nn.BatchNorm2d(ch_out)
        
        self.extra = nn.Sequential()
        if ch_out != ch_in:
            self.extra = nn.Sequential(
                nn.Conv2d(ch_in,ch_out, kernel_size = 1, stride = 1),
                nn.BatchNorm2d(ch_out)
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.extra(x) + out
        return out

In [69]:
class Net(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(Net, self).__init__()
        self.net = nn.Sequential(ResBlk(ch_in, ch_out),
                                 nn.ReLU(),
                                 nn.Linear()
                                )
    def forward(self, x):
        return self.net(x)

tensor([[ 4.,  6.],
        [ 8., 10.]])

## Pytorch中数据操作和预处理

In [11]:
import torch
import torch.utils.data as Data
from sklearn.datasets import load_boston, load_iris
import numpy as np

boston_X, boston_y = load_boston(return_X_y = True)

In [12]:
train_xt = torch.from_numpy(boston_X.astype(np.float32))
train_yt = torch.from_numpy(boston_y.astype(np.float32))

In [14]:
train_data = Data.TensorDataset(train_xt, train_yt)
train_loader = Data.DataLoader(dataset = train_data, batch_size = 64, shuffle = True, num_workers = 1)

for batch_idx, (data, target) in enumerate(train_loader):
    if batch_idx > 0:
        break
    print('data_shaoe:',data.shape)
    print('target_shape:', target.shape)

data_shaoe: torch.Size([64, 13])
target_shape: torch.Size([64])


In [15]:
iris_X, iris_y = load_iris(return_X_y = True)
print(iris_X.dtype)

float64


In [51]:
train_xt = torch.from_numpy(iris_X.astype(np.float32))
train_yt = torch.from_numpy(iris_y.astype(np.int64))

train_data = Data.TensorDataset(train_xt, train_yt)
train_loader = Data.DataLoader(dataset = train_data, batch_size = 10, shuffle = True, num_workers = 1)

for batch_idx, (data, target) in enumerate(train_loader):
    if batch_idx > 0:
        break;
    print(data.view(-1,4))
    print(data.dtype)
    print(target.shape)
    print(target.dtype)

tensor([[5.1000, 3.8000, 1.9000, 0.4000],
        [5.0000, 3.4000, 1.6000, 0.4000],
        [5.6000, 2.9000, 3.6000, 1.3000],
        [5.2000, 2.7000, 3.9000, 1.4000],
        [5.1000, 3.8000, 1.6000, 0.2000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [6.0000, 2.9000, 4.5000, 1.5000],
        [5.1000, 3.8000, 1.5000, 0.3000],
        [5.6000, 2.7000, 4.2000, 1.3000],
        [4.7000, 3.2000, 1.6000, 0.2000]])
torch.float32
torch.Size([10])
torch.int64


### 图像数据

In [67]:
import torch
import torch.utils.data as Data
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

In [82]:
train_data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224), #切成224*224
    transforms.RandomHorizontalFlip(), #p=0.5翻转
    transforms.ToTensor()
])
train_data_dir = '../../cv/insect_classification_pytorch/dataset/train/'
train_data = ImageFolder(train_data_dir, train_data_transforms)
train_data_loader = Data.DataLoader(dataset = train_data, batch_size = 4, shuffle = True, num_workers = 1)

for batch_idx,(data, target) in enumerate(train_data_loader):
    if batch_idx > 0:
        break
    print(data.shape)
    print(target.shape)
    print(data)

torch.Size([4, 3, 224, 224])
torch.Size([4])
tensor([[[[0.4863, 0.4667, 0.5098,  ..., 0.3020, 0.3216, 0.3373],
          [0.5176, 0.4941, 0.4941,  ..., 0.3098, 0.3176, 0.3333],
          [0.5608, 0.5529, 0.5333,  ..., 0.3059, 0.3098, 0.3176],
          ...,
          [0.2314, 0.2118, 0.1804,  ..., 0.2706, 0.2196, 0.1765],
          [0.2157, 0.2078, 0.1922,  ..., 0.1333, 0.0941, 0.1020],
          [0.2078, 0.1961, 0.1843,  ..., 0.0471, 0.0549, 0.0863]],

         [[0.4235, 0.4000, 0.4431,  ..., 0.3137, 0.3333, 0.3451],
          [0.4549, 0.4314, 0.4392,  ..., 0.3176, 0.3255, 0.3373],
          [0.4863, 0.4863, 0.4745,  ..., 0.3176, 0.3176, 0.3255],
          ...,
          [0.2314, 0.2118, 0.1882,  ..., 0.2824, 0.2235, 0.1569],
          [0.2157, 0.2078, 0.1961,  ..., 0.1529, 0.1020, 0.0784],
          [0.2118, 0.2039, 0.1961,  ..., 0.0392, 0.0471, 0.0745]],

         [[0.3725, 0.3686, 0.4078,  ..., 0.2863, 0.3059, 0.3137],
          [0.4000, 0.4000, 0.3882,  ..., 0.2941, 0.3020, 0.3137